In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%reload_ext autoreload
%autoreload 2

In [2]:
#Fraction of load used to downscale total LADWP EE annual savings. Currently using fraction of total LADWP load at receiving stations Q, C, Halldale.
pct_total_res_load = 0.0218336
pct_total_comm_load = 0.0707579

#Fraction of residential load that is SF vs MF in FCZ16 area of LADWP.
pct_mf_total_res_load = 0.3744324
pct_sf_total_res_load = 0.6255676

#### Read in normalized load shapes.

In [3]:
loadshapes_normalized = pd.read_csv('data/gen_profiles.csv')

#### For commercial and residential sectors, get weights for each building type and end use based on energy consumption in 2021.

In [4]:
comm_energy_use = pd.read_csv('data/CEC EE load shapes/Com-ladwp-2021/Com energy-ladwp-2021.csv')
comm_energy_use = comm_energy_use[comm_energy_use['End.Use']!='Residual']
comm_energy_use_fcz16 = comm_energy_use[comm_energy_use['FCZ']=='FCZ16']
comm_energy_use_fcz16['enduse_sum'] = comm_energy_use_fcz16.groupby(['End.Use'])['2021'].transform('sum')
comm_energy_use_fcz16['bldg_type_weight']= comm_energy_use_fcz16['2021']/comm_energy_use_fcz16['enduse_sum']
comm_weights = comm_energy_use_fcz16

<ipython-input-4-59b42a777260>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comm_energy_use_fcz16['enduse_sum'] = comm_energy_use_fcz16.groupby(['End.Use'])['2021'].transform('sum')
<ipython-input-4-59b42a777260>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comm_energy_use_fcz16['bldg_type_weight']= comm_energy_use_fcz16['2021']/comm_energy_use_fcz16['enduse_sum']


In [5]:
comm_weights

,Sector,PA,FCZ,BType_Des,End.Use,Scen,AAEE.Case,2021,enduse_sum,bldg_type_weight
0,Commercial,LADWP,FCZ16,College,Cooking,Mid,Base,1.086075,18.129334,0.059907
1,Commercial,LADWP,FCZ16,College,Cooling,Mid,Base,52.748685,633.046744,0.083325
2,Commercial,LADWP,FCZ16,College,Heating,Mid,Base,11.897406,111.908231,0.106314
3,Commercial,LADWP,FCZ16,College,IndoorLighting,Mid,Base,45.209568,1094.558458,0.041304
4,Commercial,LADWP,FCZ16,College,Miscellaneous,Mid,Base,63.514991,1011.499656,0.062793
...,...,...,...,...,...,...,...,...,...,...
115,Commercial,LADWP,FCZ16,Warehouse,OfficeEquipment,Mid,Base,1.473107,177.924683,0.008279
116,Commercial,LADWP,FCZ16,Warehouse,OutdoorLighting,Mid,Base,7.289183,276.978898,0.026317
117,Commercial,LADWP,FCZ16,Warehouse,Refrigeration,Mid,Base,0.000000,250.786862,0.000000
119,Commercial,LADWP,FCZ16,Warehouse,Ventilation,Mid,Base,6.015613,367.426970,0.016372


#### Combine dishwasher energy use with "water heating - other". Keep washer end-use separate.

In [6]:
res_energy_use = pd.read_csv('data/CEC EE load shapes/Res-ladwp-2021/CEC_res_annual_energy_LADWP_2021.csv')
res_energy_use = res_energy_use[res_energy_use['End.Use']!='Residual']
res_energy_use_fcz16 = res_energy_use[res_energy_use['FCZ']=='FCZ16']
sf_inds = res_energy_use_fcz16['BType_Des']=='SINGLEFAMILY'
mf_inds = res_energy_use_fcz16['BType_Des']=='MULTIFAMILY'

waterheater_dishwasher_inds = res_energy_use_fcz16['End.Use']=='WaterHeating-Dishwasher'
waterheater_other_inds = res_energy_use_fcz16['End.Use']=='WaterHeater-AllOther'
waterheater_washer_inds = res_energy_use_fcz16['End.Use']=='WaterHeating-Washer'

refrig_inds = res_energy_use_fcz16['End.Use']=='Refrigerator'
freezer_inds = res_energy_use_fcz16['End.Use']=='Freezer'

dishwasher_inds = res_energy_use_fcz16['End.Use']=='Dishwasher'
dryer_inds = res_energy_use_fcz16['End.Use']=='Dryer'
washer_inds = res_energy_use_fcz16['End.Use']=='Washer'


res_energy_use_fcz16.loc[sf_inds & waterheater_other_inds,'2021 (GWh)'] = res_energy_use_fcz16.loc[sf_inds & waterheater_other_inds,'2021 (GWh)'].item()+res_energy_use_fcz16.loc[sf_inds & waterheater_dishwasher_inds,'2021 (GWh)'].item()+res_energy_use_fcz16.loc[sf_inds & waterheater_washer_inds,'2021 (GWh)'].item()
res_energy_use_fcz16.loc[mf_inds & waterheater_other_inds,'2021 (GWh)'] = res_energy_use_fcz16.loc[mf_inds & waterheater_other_inds,'2021 (GWh)'].item()+res_energy_use_fcz16.loc[mf_inds & waterheater_dishwasher_inds,'2021 (GWh)'].item()+res_energy_use_fcz16.loc[mf_inds & waterheater_washer_inds,'2021 (GWh)'].item()
res_energy_use_fcz16.loc[sf_inds & refrig_inds,'2021 (GWh)'] = res_energy_use_fcz16.loc[sf_inds & refrig_inds,'2021 (GWh)'].item()+res_energy_use_fcz16.loc[sf_inds & freezer_inds,'2021 (GWh)'].item()
res_energy_use_fcz16.loc[mf_inds & refrig_inds,'2021 (GWh)'] = res_energy_use_fcz16.loc[mf_inds & refrig_inds,'2021 (GWh)'].item()+res_energy_use_fcz16.loc[mf_inds & freezer_inds,'2021 (GWh)'].item()

res_energy_use_fcz16.loc[sf_inds & dishwasher_inds,'2021 (GWh)'] = res_energy_use_fcz16.loc[sf_inds & dishwasher_inds,'2021 (GWh)']+res_energy_use_fcz16.loc[sf_inds & dryer_inds,'2021 (GWh)'].item()+res_energy_use_fcz16.loc[sf_inds & washer_inds,'2021 (GWh)'].item()
res_energy_use_fcz16.loc[mf_inds & dishwasher_inds,'2021 (GWh)'] = res_energy_use_fcz16.loc[mf_inds & dishwasher_inds,'2021 (GWh)']+res_energy_use_fcz16.loc[mf_inds & dryer_inds,'2021 (GWh)'].item()+res_energy_use_fcz16.loc[mf_inds & washer_inds,'2021 (GWh)'].item()

res_energy_use_fcz16 =res_energy_use_fcz16[res_energy_use_fcz16['End.Use']!='WaterHeating-Dishwasher']
res_energy_use_fcz16 =res_energy_use_fcz16[res_energy_use_fcz16['End.Use']!='WaterHeating-Washer']
res_energy_use_fcz16 =res_energy_use_fcz16[res_energy_use_fcz16['End.Use']!='Freezer']
res_energy_use_fcz16 =res_energy_use_fcz16[res_energy_use_fcz16['End.Use']!='Dryer']
res_energy_use_fcz16 =res_energy_use_fcz16[res_energy_use_fcz16['End.Use']!='Washer']


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1745: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [7]:
res_energy_use_fcz16['enduse_sum'] = res_energy_use_fcz16.groupby(['End.Use'])['2021 (GWh)'].transform('sum')
res_energy_use_fcz16['bldg_type_weight']= res_energy_use_fcz16['2021 (GWh)']/res_energy_use_fcz16['enduse_sum']
res_weights = res_energy_use_fcz16

In [7]:
comm_building_types = pd.DataFrame(comm_weights['BType_Des'].unique().tolist())

In [8]:
res_building_types = pd.DataFrame(res_weights['BType_Des'].unique().tolist())

#### Get annual savings from 2020-2030 for mid-mid scenario.

In [10]:
annual_ee_savings = pd.read_csv('data/CEC EE load shapes/CEC_LADWP_AAEE_annualSavings_endUse.csv')
annual_ee_savings_ladwp = annual_ee_savings[annual_ee_savings['Planning Area']=='LADWP']
annual_ee_savings_ladwp = annual_ee_savings_ladwp[annual_ee_savings_ladwp['Units']=='GWh']
annual_ee_savings_midMid = annual_ee_savings_ladwp[annual_ee_savings_ladwp['Scenario Number']==3]

In [12]:
annual_ee_savings_midMid

,Scenario Number,Scenario Name,Planning Area,Sector,End Use,Units,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030
936,3,Mid-Mid,LADWP,AGWP,MachDr,GWh,0,0.000000,0.018981,0.043622,0.070010,0.096917,0.116705,0.134460,0.152246,0.171432,0.192980,0.212647
939,3,Mid-Mid,LADWP,Commercial,Behavior,GWh,0,0.000000,74.597615,74.597615,74.597615,74.597615,74.597615,74.597615,74.597615,74.597615,74.597615,74.597615
940,3,Mid-Mid,LADWP,Commercial,Cooking,GWh,0,0.572206,0.888207,1.166246,1.425061,1.676013,1.923275,2.173972,2.375051,1.879838,1.925295,2.048563
942,3,Mid-Mid,LADWP,Commercial,HVAC - Cooling,GWh,0,44.408377,90.141720,139.672189,194.315054,254.091353,318.509431,386.359589,455.510730,523.721735,588.934740,656.116813
943,3,Mid-Mid,LADWP,Commercial,HVAC - General,GWh,0,3.603269,6.860106,9.798422,19.648943,29.119480,38.130157,49.108369,59.619647,69.860520,79.757725,89.250617
945,3,Mid-Mid,LADWP,Commercial,HVAC - Heating,GWh,0,-0.151054,-0.450633,-0.908347,-1.532935,-2.323460,-3.275860,-4.394600,-5.687596,-7.160301,-8.818355,-10.642215
947,3,Mid-Mid,LADWP,Commercial,Lighting - General,GWh,0,104.436347,167.051812,232.572740,300.522744,344.605029,375.926886,391.849646,386.490436,369.351634,343.425132,326.714654
949,3,Mid-Mid,LADWP,Commercial,Lighting - Indoor Equipment,GWh,0,0.032948,0.065884,0.098727,0.131224,0.163563,0.195498,0.227213,0.258355,0.288565,0.317015,0.342378
951,3,Mid-Mid,LADWP,Commercial,Lighting - Outdoor,GWh,0,0.032948,0.065884,0.098727,0.158593,0.218831,0.279081,0.338990,0.398275,0.456697,0.513363,0.567028
953,3,Mid-Mid,LADWP,Commercial,MachDr,GWh,0,3.335604,6.795409,10.896002,17.078023,24.026843,31.477846,39.220096,46.982090,54.672387,62.208514,69.777885


#### Scale down commercial and residential EE savings to RS Q, C, Halldale region.

In [10]:
comm_aaee_savings = annual_ee_savings_midMid[annual_ee_savings_midMid['Sector']=='Commercial']
year_cols = comm_aaee_savings.columns[6:]

In [11]:
comm_aaee_savings[year_cols] = comm_aaee_savings[year_cols]*pct_total_comm_load

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [12]:
res_aaee_savings = annual_ee_savings_midMid[annual_ee_savings_midMid['Sector']=='Residential']
year_cols = res_aaee_savings.columns[6:]
res_aaee_savings[year_cols] = res_aaee_savings[year_cols]*pct_total_res_load

In [13]:
res_end_use_map = { 
    'HVAC - Cooling':'Cooling',
    'HVAC - Heating': 'Heating',
    'Lighting - General': 'Lighting', 
    'Miscellaneous': 'Miscellaneous',
    'Plug Load - Appliances':'Dishwasher',
    'Plug Load - Consumer Electronics':'Television', 
    'Refrig/Freezer':'Refrigerator',
    'Water Heating':'WaterHeater-AllOther' 
}

In [14]:
comm_end_use_map = {
    'Cooking': 'Cooking',
    'HVAC - Cooling':'Cooling',
    'HVAC - Heating':'Heating',
    'Lighting - Indoor Equipment':'IndoorLighting',  
    'Miscellaneous':'Miscellaneous',
    'Office Equipment':'OfficeEquipment', 
    'Lighting - Outdoor':'OutdoorLighting', 
    'Refrigeration':'Refrigeration',
    'HVAC - General':'Ventilation',
    'Water Heating':'WaterHeating'
}

#### Expand comm_aaee_savings df to include rows for distinct building types.

In [15]:
comm_aaee_savings_bldgtype_enduse = comm_aaee_savings.copy()
comm_aaee_savings_bldgtype_enduse[year_cols]=np.nan

In [16]:
r = np.arange(len(comm_aaee_savings_bldgtype_enduse)).repeat(len(comm_building_types))
comm_building_types_df = pd.concat([comm_building_types]*len(comm_aaee_savings_bldgtype_enduse),ignore_index=True)
comm_eesavings_bldgtype_enduse = pd.DataFrame(comm_aaee_savings_bldgtype_enduse.values[r], columns = comm_aaee_savings_bldgtype_enduse.columns).join(comm_building_types_df)
comm_eesavings_bldgtype_enduse.rename(columns={0:'building_type'}, inplace=True)

In [17]:
comm_eesavings_bldgtype_enduse = comm_eesavings_bldgtype_enduse[['Scenario Number', 'Scenario Name', 'Planning Area', 'Sector',
       'building_type','End Use', 'Units', '2019', '2020', '2021', '2022', '2023', '2024',
       '2025', '2026', '2027', '2028', '2029', '2030']]

#### Expand res_aaee_savings df to include rows for distinct building types.

In [18]:
res_aaee_savings_bldgtype_enduse = res_aaee_savings.copy()
res_aaee_savings_bldgtype_enduse[year_cols]=np.nan

In [19]:
n = np.arange(len(res_aaee_savings_bldgtype_enduse)).repeat(len(res_building_types))
res_building_types_df = pd.concat([res_building_types]*len(res_aaee_savings_bldgtype_enduse),ignore_index=True)
res_eesavings_bldgtype_enduse = pd.DataFrame(res_aaee_savings_bldgtype_enduse.values[n], columns = res_aaee_savings_bldgtype_enduse.columns).join(res_building_types_df)
res_eesavings_bldgtype_enduse.rename(columns={0:'building_type'}, inplace=True)

In [20]:
res_eesavings_bldgtype_enduse = res_eesavings_bldgtype_enduse[['Scenario Number', 'Scenario Name', 'Planning Area', 'Sector',
       'building_type','End Use', 'Units', '2019', '2020', '2021', '2022', '2023', '2024',
       '2025', '2026', '2027', '2028', '2029', '2030']]

#### Apply comm building type weights to each end use savings to get the savings by building type and end use.

In [21]:
for index,row in comm_eesavings_bldgtype_enduse.iterrows():
    bldg_type = row['building_type']
    end_use = row['End Use']
    if end_use in comm_end_use_map.keys():
        weights_enduse = comm_end_use_map[end_use]
        enduse_weight_inds = comm_weights['End.Use']==weights_enduse
        bldgtype_weight_inds = comm_weights['BType_Des']==bldg_type
        weight = comm_weights[enduse_weight_inds & bldgtype_weight_inds]['bldg_type_weight'].item()
        
        for year in comm_eesavings_bldgtype_enduse.columns[7:]:
            enduse_inds = comm_aaee_savings['End Use']==end_use
            enduse_savings_year = comm_aaee_savings[enduse_inds][year].item()

            enduse_bldgtype_savings = enduse_savings_year*weight

            comm_eesavings_bldgtype_enduse.loc[index,year]=enduse_bldgtype_savings
        
    else:
        comm_eesavings_bldgtype_enduse.drop([index], inplace=True)
    

#### Apply res building type weights to each end use savings to get the savings by building type and end use.

In [22]:
for index,row in res_eesavings_bldgtype_enduse.iterrows():
    bldg_type = row['building_type']
    end_use = row['End Use']
    if end_use in res_end_use_map.keys():
        weights_enduse = res_end_use_map[end_use]
        enduse_weight_inds = res_weights['End.Use']==weights_enduse
        bldgtype_weight_inds = res_weights['BType_Des']==bldg_type
        weight = res_weights[enduse_weight_inds & bldgtype_weight_inds]['bldg_type_weight'].item()
        
        for year in res_eesavings_bldgtype_enduse.columns[7:]:
            enduse_inds = res_aaee_savings['End Use']==end_use
            enduse_savings_year = res_aaee_savings[enduse_inds][year].item()

            enduse_bldgtype_savings = enduse_savings_year*weight

            res_eesavings_bldgtype_enduse.loc[index,year]=enduse_bldgtype_savings
        
    else:
        res_eesavings_bldgtype_enduse.drop([index], inplace=True)
    

#### Divide annual savings for each commercial buildingtype/enduse by the total sum of the normalized load shape to get max MW for that resource.

In [23]:
comm_end_use_savings_to_loadshapes = {
    'Cooking': 'Cooking',
    'HVAC - Cooling': 'Cooling',
    'HVAC - General': 'Ventilation',
    'HVAC - Heating': 'Heating',
    'Lighting - Indoor Equipment': 'IndoorLightingEff',
    'Lighting - Outdoor': 'OutdoorLighting',
    'Miscellaneous': 'Miscellaneous',
    'Office Equipment': 'OfficeEquipment',
    'Refrigeration': 'Refrigeration',
    'Water Heating': 'WaterHeating'
}

In [24]:
ee_resource_potential = pd.DataFrame(columns=year_cols)

In [25]:
for index, row in comm_eesavings_bldgtype_enduse.iterrows():
    sector = row['Sector']
    bldg_type = row['building_type']
    end_use = row['End Use']
    loadshapes_end_use = comm_end_use_savings_to_loadshapes[end_use]
    loadshape_columns_name = 'FCZ7.'+'Commercial.'+bldg_type+'.'+loadshapes_end_use
    column = loadshapes_normalized[loadshape_columns_name]
        
    annual_loadshape_sum = column.sum()
    
    ee_resource_potential.loc[index,'resource']=loadshape_columns_name
    
    for year in comm_eesavings_bldgtype_enduse.columns[7:]:
        max_annual_savings_mwh = row[year]*1000
        
        max_mw = max_annual_savings_mwh/annual_loadshape_sum
        
        ee_resource_potential.loc[index,year] = max_mw

#### Divide annual savings for each residential buildingtype/enduse by the total sum of the normalized load shape to get max MW for that resource.

In [26]:
res_end_use_savings_to_loadshapes = {
    'HVAC - Cooling':'CoolingEff',
    'HVAC - Heating': 'Heating',
    'Lighting - General': 'ResLightingEff', 
    'Miscellaneous': 'Miscellaneous',
    'Plug Load - Appliances':['Washer','Dryer'],
    'Refrig/Freezer':'Refrigerator',
    'Water Heating':'WaterHeating' 
}

In [27]:
res_eesavings_bldgtype_enduse = res_eesavings_bldgtype_enduse[res_eesavings_bldgtype_enduse['End Use']!='Plug Load - Consumer Electronics']

In [28]:
for index, row in res_eesavings_bldgtype_enduse.iterrows():
    sector = row['Sector']
    bldg_type = row['building_type']
    end_use = row['End Use']

    loadshapes_end_use = res_end_use_savings_to_loadshapes[end_use]

    #For plug load — appliances, split savings between washer and dryer.
    if isinstance(loadshapes_end_use,list):
        for i in loadshapes_end_use:
            loadshape_columns_name = 'FCZ7.'+'Residential.'+bldg_type+'.'+str(i)
            column = loadshapes_normalized[loadshape_columns_name]
        
            annual_loadshape_sum = column.sum()
    
            ee_resource_potential.loc[index,'resource']=loadshape_columns_name
    
            for year in res_eesavings_bldgtype_enduse.columns[7:]:
                max_annual_savings_mwh = row[year]*1000/2   
                max_mw = max_annual_savings_mwh/annual_loadshape_sum

                ee_resource_potential.loc[index,year] = max_mw
        
    else:  
        loadshape_columns_name = 'FCZ7.'+'Residential.'+bldg_type+'.'+loadshapes_end_use
        column = loadshapes_normalized[loadshape_columns_name]

        annual_loadshape_sum = column.sum()

        ee_resource_potential.loc[index,'resource']=loadshape_columns_name

        for year in res_eesavings_bldgtype_enduse.columns[7:]:
            max_annual_savings_mwh = row[year]*1000    
            max_mw = max_annual_savings_mwh/annual_loadshape_sum

            ee_resource_potential.loc[index,year] = max_mw

In [29]:
ee_resource_potential['units']='MW'

In [30]:
years_cols = ['2019', '2020', '2021', '2022', '2023', '2024', '2025', '2026', '2027', '2028', '2029', '2030'] 
numeric_data = ee_resource_potential[years_cols]

In [31]:
numeric_data[numeric_data <0]=0

<ipython-input-31-f84d69028c1f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_data[numeric_data <0]=0
/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3089: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


In [32]:
numeric_data['resource']= ee_resource_potential['resource']
numeric_data['units']= 'MW'
numeric_data.to_csv('data/ee_resource_potential_2020_2030.csv')

<ipython-input-32-c882411cffa7>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  numeric_data['resource']= ee_resource_potential['resource']


#### Calculate annual savings for each EE measure in 2030 to determine which can be dropped. (multiply resource potential in 2030 by summed loadshape. 

In [47]:
resource_potential_2030 = numeric_data[['2030', 'resource']]
resource_potential_2030.rename(columns={'2030':'resource_potential_(MW, 2030)'}, inplace=True)

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4295: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [48]:
loadshapes_normalized_summed = loadshapes_normalized.sum().reset_index()
loadshapes_normalized_summed.rename(columns={'index':'resource', '0':'annual savings (mwh/mw)'},inplace=True)

In [49]:
merged = resource_potential_2030.merge(loadshapes_normalized_summed, on='resource')

In [55]:
merged['annual_savings_2030']= merged['resource_potential_(MW, 2030)']*merged[0]

In [58]:
merged.sort_values(by=['annual_savings_2030'], inplace=True)

In [61]:
merged.to_csv('data/ee_annualSavings_2030.csv')

In [78]:
#annual_ee_savings_midMid_bySector = annual_ee_savings_midMid.groupby(['Sector','End Use']).sum()

In [79]:
#annual_ee_savings_midMid_bySector.loc['Residential']

,Scenario Number,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030
End Use,,,,,,,,,,,,,
Behavior,3,0,23.800789,74.893336,76.402947,77.935058,79.501693,81.139050,82.810129,84.515624,86.256245,88.032714,89.773696
HVAC - Cooling,3,0,5.200210,13.215910,25.512038,44.265977,72.594041,108.291598,151.727974,199.061302,242.993282,278.539669,319.979528
HVAC - General,3,0,2.185179,4.508558,6.752640,10.552760,14.261195,17.729134,21.225644,24.505464,27.703057,30.763487,33.665600
HVAC - Heating,3,0,-0.493236,-1.506128,-3.016486,-4.972535,-7.313523,-10.010235,-13.189268,-16.861344,-21.036205,-25.723744,-30.880037
Lighting - General,3,0,90.327646,133.300013,157.637192,180.404657,207.409963,240.514275,278.459956,314.287982,341.479902,360.912149,388.497870
Miscellaneous,3,0,12.098138,25.397213,40.016843,53.757170,70.258038,88.236357,101.292895,102.634175,95.706891,83.424082,77.921801
Plug Load - Appliances,3,0,0.000000,3.574274,11.097659,18.193265,25.173799,31.795236,37.246525,38.980693,40.253330,40.887751,41.417811
Plug Load - Consumer Electronics,3,0,0.419166,0.754808,1.061201,1.359803,1.657336,1.941106,1.797372,1.791290,1.856247,1.900133,1.919035
Refrig/Freezer,3,0,69.962649,123.737199,173.647629,195.942459,201.497790,201.172832,215.209212,231.089857,248.346628,266.335120,283.269688


In [80]:
annual_ee_savings_midMid_fcz16_sf = annual_ee_savings_midMid_bySector.loc['Residential']*load_share_fcz16_sf

In [82]:
annual_ee_savings_midMid_fcz16_mf = annual_ee_savings_midMid_bySector.loc['Residential']*load_share_fcz16_mf

In [83]:
ee_loadshapes = pd.read_csv('data/CEC EE load shapes/Res-ladwp-2021/CEC_res_loadshapes_EE_fz7_2021.csv')

In [85]:
ee_loadshapes.columns

Index(['Unnamed: 0', 'DatesAll', 'FCZ7.Residential.MULTIFAMILY.CoolingEff',
       'FCZ7.Residential.MULTIFAMILY.Dryer',
       'FCZ7.Residential.MULTIFAMILY.Heating',
       'FCZ7.Residential.MULTIFAMILY.HeatPumpEff',
       'FCZ7.Residential.MULTIFAMILY.Miscellaneous',
       'FCZ7.Residential.MULTIFAMILY.Refrigerator',
       'FCZ7.Residential.MULTIFAMILY.ResLightingEff',
       'FCZ7.Residential.MULTIFAMILY.ResOccsensorEff',
       'FCZ7.Residential.MULTIFAMILY.Washer',
       'FCZ7.Residential.MULTIFAMILY.WaterHeating',
       'FCZ7.Residential.SINGLEFAMILY.CoolingEff',
       'FCZ7.Residential.SINGLEFAMILY.Dryer',
       'FCZ7.Residential.SINGLEFAMILY.Heating',
       'FCZ7.Residential.SINGLEFAMILY.HeatPumpEff',
       'FCZ7.Residential.SINGLEFAMILY.Miscellaneous',
       'FCZ7.Residential.SINGLEFAMILY.Refrigerator',
       'FCZ7.Residential.SINGLEFAMILY.ResLightingEff',
       'FCZ7.Residential.SINGLEFAMILY.ResOccsensorEff',
       'FCZ7.Residential.SINGLEFAMILY.Washer',
       

In [24]:
# res_annual_energy = pd.read_csv('data/CEC EE load shapes/Res-ladwp-2021/CEC_res_annual_energy_LADWP_2021.csv')
# res_annual_energy_enduse = res_annual_energy[res_annual_energy['End.Use']!='Residual']
# load_split_fcz = res_annual_energy_enduse.groupby(['FCZ','BType_Des']).sum()
# fcz16 = load_split_fcz.loc['FCZ16']
#load_share_fcz16_sf = fcz16.loc['SINGLEFAMILY'].item() / fcz16.sum().item()
#load_share_fcz16_mf = fcz16.loc['MULTIFAMILY'].item() / fcz16.sum().item()